___

<a href='http://www.pieriandata.com'><img src='../Pierian_Data_Logo.png'/></a>
___
<center><em>Авторские права принадлежат Pierian Data Inc.</em></center>
<center><em>Для дополнительной информации посетите наш сайт <a href='http://www.pieriandata.com'>www.pieriandata.com</a></em></center>

# Кросс-валидация - введение

В этой серии лекций мы подробно рассмотрим различные методы кросс-валидации, а также обсудим общую идею кросс-валидации. Хорошую документацию по этой теме можно почитать вот здесь: https://scikit-learn.org/stable/modules/cross_validation.html

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Пример данных

In [2]:
df = pd.read_csv("../DATA/Advertising.csv")

In [3]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


----
----
----
## Разбиваем данные на обучающий и тестовый наборы данных - Train | Test Split

0. Очищаем и масштабируем данные X и y (при необходимости)
1. Разбиваем данные на обучающий и тестовый наборы данных - как для X, так и для y
2. Обучаем объект Scaler на обучающих данных X
3. Применяем масштабирование (scale) для тестовых данных X
4. Создаём модель
5. Обучаем модель на обучающих данных X
6. Оцениваем модель на тестовых данных X (создавая предсказания и сравнивая их с Y_test)
7. Уточняем параметры модели, повторяя шаги 5 и 6

In [5]:
## Создаём X и y
X = df.drop('sales',axis=1)
y = df['sales']

# Разбиение на TRAIN и TEST
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# Масштабирование данных - SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

**Создаём модель**

In [6]:
from sklearn.linear_model import Ridge

In [7]:
# Мы осознанно указываем неудачное значение Alpha!
model = Ridge(alpha=100)

In [8]:
model.fit(X_train,y_train)

Ridge(alpha=100)

In [9]:
y_pred = model.predict(X_test)

**Оценка модели (evaluation)**

In [10]:
from sklearn.metrics import mean_squared_error

In [11]:
mean_squared_error(y_test,y_pred)

7.34177578903413

**Уточняем параметры и заново оцениваем модель**

In [12]:
model = Ridge(alpha=1)

In [13]:
model.fit(X_train,y_train)

Ridge(alpha=1)

In [14]:
y_pred = model.predict(X_test)

**Повторная оценка**

In [15]:
mean_squared_error(y_test,y_pred)

2.319021579428751

Намного лучше! Мы можем повторять эти шаги до тех пор, пока не получим хорошие метрики производительности. (Ранее мы видели, как для этой цели использовать RidgeCV, но здесь мы можем обобщить процесс кросс-валидации для случая любой модели).

----
----
----
## Обучение - валидация - тестирование (Train | Validation | Test Split Procedure)

Этот подход ещё называют методом с задержкой ("hold-out"), потому что здесь мы не настраиваем параметры модели на финальном тестовом наборе данных, а используем тестовые данные *только* для оценки финального результата.

0. Очищаем и масштабируем данные X и y (при необходимости)
1. Разбиваем данные на обучающий, оценочный и тестовый наборы данных - как для X, так и для y
2. Обучаем объект Scaler на обучающих данных X
3. Масштабируем (scale) оценочные данные X
4. Создаём модель
5. Обучаем модель на обучающих данных X
6. Оцениваем модель на оценочных данных X (создавая предсказания и сравнивая их с Y_eval)
7. Уточняем параметры модели, повторяя шаги 5 и 6
8. Вычисляем финальные метрики на тестовом наборе данных (после этого уже нельзя возвращаться и делать уточнения!)

In [16]:
## Создаём X и y
X = df.drop('sales',axis=1)
y = df['sales']

In [17]:
###############################################################################################
#### Вызываем SPLIT дважды! Здесь мы создаём три набора данных - TRAIN, VALIDATION и TEST  #########
###############################################################################################
from sklearn.model_selection import train_test_split

# 70% данных определяем в обучающий набор, остальные 30% откладываем в сторону
X_train, X_OTHER, y_train, y_OTHER = train_test_split(X, y, test_size=0.3, random_state=101)

# Оставшиеся 30% разбиваем на оценочный и тестовый наборы данных
# Каждый будет по 15% от исходного набора данных 
X_eval, X_test, y_eval, y_test = train_test_split(X_OTHER, y_OTHER, test_size=0.5, random_state=101)

In [18]:
# Масштабируем данные (SCALE)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_eval = scaler.transform(X_eval)
X_test = scaler.transform(X_test)

**Создаём модель**

In [19]:
from sklearn.linear_model import Ridge

In [20]:
# Мы осознанно указываем неудачное значение Alpha!
model = Ridge(alpha=100)

In [21]:
model.fit(X_train,y_train)

Ridge(alpha=100, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [22]:
y_eval_pred = model.predict(X_eval)

**Оценка модели (Evaluation)**

In [23]:
from sklearn.metrics import mean_squared_error

In [24]:
mean_squared_error(y_eval,y_eval_pred)

7.320101458823871

**Уточняем параметры и повторно оцениваем модель**

In [25]:
model = Ridge(alpha=1)

In [26]:
model.fit(X_train,y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

In [27]:
y_eval_pred = model.predict(X_eval)

**Ещё раз оцениваем модель**

In [28]:
mean_squared_error(y_eval,y_eval_pred)

2.383783075056986

**Вычисляем финальные метрики на тестовом наборе данных (после этого уже нельзя возвращаться и делать уточнения!)**

In [29]:
y_final_test_pred = model.predict(X_test)

In [30]:
mean_squared_error(y_test,y_final_test_pred)

2.254260083800517

----
----
----
## Кросс-валидация с помощью cross_val_score

----

<img src="grid_search_cross_validation.png">

----

In [44]:
## Создаём X и y
X = df.drop('sales',axis=1)
y = df['sales']

# Разбиение на TRAIN и TEST
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# Масштабирование данных (SCALE)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [45]:
model = Ridge(alpha=100)

In [46]:
from sklearn.model_selection import cross_val_score

In [48]:
# Варианты оценки модели:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_val_score(model,X_train,y_train,
                         scoring='neg_mean_squared_error',cv=5)

In [49]:
scores

array([ -9.32552967,  -4.9449624 , -11.39665242,  -7.0242106 ,
        -8.38562723])

In [50]:
# Среднее значение MSE scores (делаем это значение положительным)
abs(scores.mean())

8.215396464543607

**Уточняем модель на основе метрик**

In [51]:
model = Ridge(alpha=1)

In [52]:
# Варианты оценки модели:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_val_score(model,X_train,y_train,
                         scoring='neg_mean_squared_error',cv=5)

In [53]:
# Среднее значение MSE scores (делаем это значение положительным)
abs(scores.mean())

3.344839296530695

**Вычисляем финальные метрики на тестовом наборе данных (после этого уже нельзя возвращаться и делать уточнения!)**

In [55]:
# Сначала нужно обучить модель!
model.fit(X_train,y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

In [56]:
y_final_test_pred = model.predict(X_test)

In [57]:
mean_squared_error(y_test,y_final_test_pred)

2.319021579428752

----
----
----

# Кросс-валидация с помощью cross_validate

Функция cross_validate отличается от cross_val_score двумя аспектами:

эта функция позволяет использовать для оценки несколько метрик;

она возвращает не только оценку на тестовом наборе (test score), но и словарь с замерами времени обучения и скоринга, а также - опционально - оценки на обучающем наборе и объекты estimator.

В случае одной метрики для оценки, когда параметр scoring является строкой string, вызываемым объектом callable или значением None, ключи словаря будут следующими:
        
        - ['test_score', 'fit_time', 'score_time']

А в случае нескольких метрик для оценки, возвращаемый словарь будет содержать следующие ключи:

    ['test_<scorer1_name>', 'test_<scorer2_name>', 'test_<scorer...>', 'fit_time', 'score_time']

return_train_score по умолчанию принимает значение False, чтобы сэкономить вычислительные ресурсы. Чтобы посчитать оценки на обучающем наборе, достаточно установить этот параметр в значение True.

In [62]:
## Создаём X и y
X = df.drop('sales',axis=1)
y = df['sales']

# Делаем разбиение на TRAIN и TEST
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# Масштабируем данные (SCALE)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [63]:
model = Ridge(alpha=100)

In [64]:
from sklearn.model_selection import cross_validate

In [72]:
# Варианты оценки модели:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_validate(model,X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error'],cv=5)

In [73]:
scores

{'fit_time': array([0.00102687, 0.00088882, 0.00099993, 0.00099945, 0.        ]),
 'score_time': array([0.00108409, 0.        , 0.        , 0.00064516, 0.00086308]),
 'test_neg_mean_absolute_error': array([-2.31243044, -1.74653361, -2.56211701, -2.01873159, -2.27951906]),
 'test_neg_mean_squared_error': array([ -9.32552967,  -4.9449624 , -11.39665242,  -7.0242106 ,
         -8.38562723]),
 'test_max_error': array([ -6.44988486,  -5.58926073, -10.33914027,  -6.61950405,
         -7.75578515])}

In [74]:
pd.DataFrame(scores)

,fit_time,score_time,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_max_error
0,0.001027,0.001084,-2.312430,-9.325530,-6.449885
1,0.000889,0.000000,-1.746534,-4.944962,-5.589261
2,0.001000,0.000000,-2.562117,-11.396652,-10.339140
3,0.000999,0.000645,-2.018732,-7.024211,-6.619504
4,0.000000,0.000863,-2.279519,-8.385627,-7.755785


In [75]:
pd.DataFrame(scores).mean()

fit_time                        0.000783
score_time                      0.000518
test_neg_mean_absolute_error   -2.183866
test_neg_mean_squared_error    -8.215396
test_max_error                 -7.350715
dtype: float64

**Уточняем модель на основе метрик**

In [76]:
model = Ridge(alpha=1)

In [77]:
# Варианты оценки модели:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_validate(model,X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error'],cv=5)

In [78]:
pd.DataFrame(scores).mean()

fit_time                        0.000901
score_time                      0.000200
test_neg_mean_absolute_error   -1.319685
test_neg_mean_squared_error    -3.344839
test_max_error                 -5.161145
dtype: float64

**Вычисляем финальные метрики на тестовом наборе данных (после этого уже нельзя возвращаться и делать уточнения!)**

In [79]:
# Сначала нужно обучить модель!
model.fit(X_train,y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

In [80]:
y_final_test_pred = model.predict(X_test)

In [81]:
mean_squared_error(y_test,y_final_test_pred)

2.319021579428752

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df = pd.read_csv('../DATA/Advertising.csv')

In [7]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [9]:
X = df.drop('sales', axis=1)
y = df['sales']

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [12]:
from sklearn.preprocessing import StandardScaler

In [13]:
scaler = StandardScaler()

In [15]:
scaler.fit(X_train)

StandardScaler()

In [16]:
X_train = scaler.transform(X_train)

In [18]:
X_test = scaler.transform(X_test)

In [20]:
from sklearn.linear_model import Ridge

In [21]:
model = Ridge(alpha=100)

In [22]:
model.fit(X_train, y_train)

Ridge(alpha=100)

In [23]:
y_pred = model.predict(X_test)

In [24]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [25]:
mean_squared_error(y_test, y_pred)

7.341775789034128

In [26]:
model_two = Ridge(alpha=1)

In [27]:
model_two.fit(X_train, y_train)

Ridge(alpha=1)

In [28]:
y_pred_two = model_two.predict(X_test)

In [29]:
mean_squared_error(y_test, y_pred_two)

2.3190215794287514

In [35]:
for i in range(31):
    mod = Ridge(alpha=i)
    mod.fit(X_train, y_train)
    y_mod = mod.predict(X_test)
    print(i, mean_squared_error(y_test, y_mod))

0 2.2987166978863796
1 2.3190215794287514
2 2.341360018625149
3 2.3656314101377887
4 2.3917401564726686
5 2.419595382593396
6 2.449110669290265
7 2.4802038039004772
8 2.5127965470932163
9 2.5468144145405716
10 2.5821864723922543
11 2.6188451455604973
12 2.656726037901746
13 2.6957677634549686
14 2.7359117879629116
15 2.7771022799634912
16 2.8192859707940086
17 2.8624120229014656
18 2.906431905898722
19 2.951299279848608
20 2.9969698852970277
21 3.0434014396116065
22 3.0905535392153154
23 3.138387567334313
24 3.186866606907059
25 3.235955358326854
26 3.285620061713614
27 3.3358284234319355
28 3.386549546592437
29 3.437753865291678
30 3.4894130823627125


In [36]:
X = df.drop('sales',axis=1)
y = df['sales']

In [40]:
X_train, X_other, y_train, y_other = train_test_split(X, y, test_size=0.3, random_state=101)

In [41]:
X_eval, X_test, y_eval, y_test = train_test_split(X_other, y_other, test_size=0.5, random_state=101)

In [42]:
len(df)

200

In [43]:
len(X_train)

140

In [44]:
len(X_eval)

30

In [45]:
len(X_test)

30

In [47]:
scaler = StandardScaler()

In [48]:
scaler.fit(X_train)

StandardScaler()

In [49]:
X_train = scaler.transform(X_train)

In [50]:
X_eval = scaler.transform(X_eval)

In [51]:
X_test = scaler.transform(X_test)

In [52]:
model_one = Ridge(alpha=100)

In [53]:
model_one.fit(X_train, y_train)

Ridge(alpha=100)

In [54]:
y_eval_pred = model_one.predict(X_eval)

In [55]:
mean_squared_error(y_eval, y_eval_pred)

7.320101458823869

In [56]:
for i in range(31):
    mod = Ridge(alpha=i)
    mod.fit(X_train, y_train)
    y_mod = mod.predict(X_eval)
    print(i, mean_squared_error(y_eval, y_mod))

0 2.3564786656264913
1 2.3837830750569853
2 2.4126999382381316
3 2.443146687715333
4 2.4750449010494306
5 2.5083200632605
6 2.5429013449725972
7 2.578721395076946
8 2.6157161468309744
9 2.6538246364014175
10 2.692988832941702
11 2.733153479368672
12 2.7742659430714967
13 2.816276075847455
14 2.8591360824154246
15 2.902800396909275
16 2.947225566800138
17 2.992370143739244
18 3.0381945808520188
19 3.084661136049992
20 3.131733780959673
21 3.179378115097506
22 3.2275612849476505
23 3.2762519076243684
24 3.3254199988241777
25 3.375036904793985
26 3.4250752380613108
27 3.4755088166904637
28 3.5263126068453143
29 3.5774626684546416
30 3.628936103790006


In [57]:
model_2 = Ridge(alpha=1)

In [59]:
model_2.fit(X_train, y_train)

Ridge(alpha=1)

In [61]:
new_pred_eval = model_2.predict(X_eval)

In [62]:
mean_squared_error(y_eval, new_pred_eval)

2.3837830750569853

In [63]:
y_final_test_pred = model_2.predict(X_test)

In [64]:
mean_squared_error(y_test, y_final_test_pred)

2.254260083800517

In [65]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [66]:
X

,TV,radio,newspaper
0,230.1,37.8,69.2
1,44.5,39.3,45.1
2,17.2,45.9,69.3
3,151.5,41.3,58.5
4,180.8,10.8,58.4
...,...,...,...
195,38.2,3.7,13.8
196,94.2,4.9,8.1
197,177.0,9.3,6.4
198,283.6,42.0,66.2


In [67]:
y

0      22.1
1      10.4
2       9.3
3      18.5
4      12.9
       ... 
195     7.6
196     9.7
197    12.8
198    25.5
199    13.4
Name: sales, Length: 200, dtype: float64

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101) # лучше указать 15 процентов

In [70]:
scaler = StandardScaler()

In [71]:
scaler.fit(X_train)

StandardScaler()

In [72]:
X_train = scaler.transform(X_train)

In [73]:
X_test = scaler.transform(X_test)

In [74]:
model = Ridge(alpha=100)

In [75]:
from sklearn.model_selection import cross_val_score

In [78]:
scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=5)

In [79]:
scores

array([ -9.32552967,  -4.9449624 , -11.39665242,  -7.0242106 ,
        -8.38562723])

In [80]:
abs(scores.mean())

8.215396464543607

In [81]:
model = Ridge(alpha=1)

In [82]:
scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=5)

In [83]:
scores

array([-3.15513238, -1.58086982, -5.40455562, -2.21654481, -4.36709384])

In [84]:
abs(scores.mean())

3.344839296530695

In [85]:
model.fit(X_train, y_train)

Ridge(alpha=1)

In [86]:
y_final_test_pred = model.predict(X_test)

In [87]:
mean_squared_error(y_test, y_final_test_pred)

2.3190215794287514

In [88]:
## Создаём X и y
X = df.drop('sales',axis=1)
y = df['sales']

# Делаем разбиение на TRAIN и TEST
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# Масштабируем данные (SCALE)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [89]:
from sklearn.model_selection import cross_validate

In [90]:
model = Ridge(alpha=100)

In [91]:
scores = cross_validate(model, X_train, y_train, scoring=['neg_mean_squared_error', 'neg_mean_absolute_error'], cv=10)

In [92]:
scores

{'fit_time': array([0.        , 0.        , 0.        , 0.00150752, 0.0010004 ,
        0.00050688, 0.00100851, 0.0010004 , 0.00238085, 0.00099945]),
 'score_time': array([0.00150466, 0.        , 0.        , 0.        , 0.        ,
        0.00100589, 0.00099993, 0.00099874, 0.00099826, 0.00100207]),
 'test_neg_mean_squared_error': array([ -6.06067062, -10.62703078,  -3.99342608,  -5.00949402,
         -9.14179955, -13.08625636,  -3.83940454,  -9.05878567,
         -9.05545685,  -5.77888211]),
 'test_neg_mean_absolute_error': array([-1.8102116 , -2.54195751, -1.46959386, -1.86276886, -2.52069737,
        -2.45999491, -1.45197069, -2.37739501, -2.44334397, -1.89979708])}

In [93]:
scores = pd.DataFrame(scores)

In [94]:
scores

,fit_time,score_time,test_neg_mean_squared_error,test_neg_mean_absolute_error
0,0.000000,0.001505,-6.060671,-1.810212
1,0.000000,0.000000,-10.627031,-2.541958
2,0.000000,0.000000,-3.993426,-1.469594
3,0.001508,0.000000,-5.009494,-1.862769
4,0.001000,0.000000,-9.141800,-2.520697
5,0.000507,0.001006,-13.086256,-2.459995
6,0.001009,0.001000,-3.839405,-1.451971
7,0.001000,0.000999,-9.058786,-2.377395
8,0.002381,0.000998,-9.055457,-2.443344
9,0.000999,0.001002,-5.778882,-1.899797


In [95]:
scores.mean(numeric_only=True)

fit_time                        0.000840
score_time                      0.000651
test_neg_mean_squared_error    -7.565121
test_neg_mean_absolute_error   -2.083773
dtype: float64

In [96]:
model = Ridge(alpha=1)

In [97]:
scores = cross_validate(model, X_train, y_train, scoring=['neg_mean_squared_error', 'neg_mean_absolute_error'], cv=10)

In [98]:
scores = pd.DataFrame(scores)

In [99]:
scores.mean()

fit_time                        0.001127
score_time                      0.000728
test_neg_mean_squared_error    -3.323018
test_neg_mean_absolute_error   -1.308467
dtype: float64